# Further explorations on the Citibike data
- Find out which stations are popular among subscribers and non-subscribers

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.linear_model import LinearRegression, RANSACRegressor
%matplotlib inline

In [2]:
# Import datasets
d1 = pd.read_csv('../data/processed/extended2/p_s_alltime_extended.csv')
d2 = pd.read_csv('../data/processed/extended2/p_ns_alltime_extended.csv')

In [3]:
# data wrangling
d1['usertype'] = 'subscriber'
d2['usertype'] = 'customer'
d1 = d1[d1['AWND'] >= 0]
d2 = d2[d2['AWND'] >= 0]
data = pd.concat([d1, d2], axis=0)
del data['Unnamed: 0']
del data['DATE']

In [4]:
data.columns

Index(['date', 'pickups', 'st_id', 'st_name', 'st_latitude', 'st_longitude',
       'closest_college', 'closest_college_distance', 'closest_subway',
       'closest_subway_distance', 'closest_theater',
       'closest_theater_distance', 'closest_museum', 'closest_museum_distance',
       'closest_park_name', 'closest_park_distance', 'PRCP', 'SNOW', 'SNWD',
       'TMAX', 'TMIN', 'AWND', 'usertype'],
      dtype='object')

In [5]:
data.head()

,date,pickups,st_id,st_name,st_latitude,st_longitude,closest_college,closest_college_distance,closest_subway,closest_subway_distance,...,closest_museum_distance,closest_park_name,closest_park_distance,PRCP,SNOW,SNWD,TMAX,TMIN,AWND,usertype
0,20130701,47,72,W 52 St & 11 Ave,40.767272,-73.993929,American Academy Mcallister Institute Of Funer...,0.449863,8th Ave & 50th St at NW corner,0.836766,...,0.648424,De Witt Clinton Park,0.190527,21.3,0.0,0.0,25.0,22.2,1.4,subscriber
1,20130701,73,79,Franklin St & W Broadway,40.719116,-74.006667,Borough Of Manhattan Comm College (Cuny),0.438224,West Broadway & Franklin St at SW corner,0.012754,...,0.470931,GREENSTREET,0.154135,21.3,0.0,0.0,25.0,22.2,1.4,subscriber
2,20130701,13,82,St James Pl & Pearl St,40.711174,-74.000165,Pace University / New York City Campus,0.390559,Centre St & Chambers St at SE corner,0.372382,...,0.590027,James Madison Plaza,0.143915,21.3,0.0,0.0,25.0,22.2,1.4,subscriber
3,20130701,19,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,Long Island University / Brooklyn Campus,0.970719,4th Ave & Pacific St at NE corner,0.176488,...,0.224781,Brooklyn Academy Of Music,0.108015,21.3,0.0,0.0,25.0,22.2,1.4,subscriber
4,20130701,47,116,W 17 St & 8 Ave,40.741776,-74.001497,New School University / Jazz And Contemporary ...,0.717327,8th Ave & 16th St at NW corner,0.062889,...,0.368229,Dr. Gertrude B. Kelly Playground,0.170808,21.3,0.0,0.0,25.0,22.2,1.4,subscriber


In [6]:
data.describe().round(2)

,date,pickups,st_id,st_latitude,st_longitude,closest_college_distance,closest_subway_distance,closest_theater_distance,closest_museum_distance,closest_park_distance,PRCP,SNOW,SNWD,TMAX,TMIN,AWND
count,927184.00,927184.00,927184.00,926658.00,926658.00,926658.00,926658.00,926658.00,926658.00,926658.00,927184.00,927184.00,927184.00,927184.00,927184.00,927184.00
mean,20151089.71,41.66,969.03,40.73,-73.98,0.66,0.28,1.12,0.66,0.15,2.70,1.34,10.23,18.39,10.28,2.33
std,11423.29,55.11,1129.51,0.03,0.02,0.42,0.23,1.27,0.63,0.03,7.61,12.26,48.83,9.91,9.19,1.01
min,20130701.00,1.00,72.00,40.52,-74.03,0.02,0.00,0.01,0.02,0.09,0.00,0.00,0.00,-9.30,-18.20,0.30
25%,20140816.00,6.00,315.00,40.71,-74.00,0.33,0.09,0.25,0.27,0.13,0.00,0.00,0.00,10.60,3.30,1.60
50%,20150924.00,19.00,430.00,40.73,-73.99,0.59,0.22,0.52,0.48,0.15,0.00,0.00,0.00,19.40,10.60,2.20
75%,20160716.00,57.00,532.00,40.75,-73.97,0.90,0.41,1.64,0.76,0.17,0.80,0.00,0.00,27.20,18.30,2.90
max,20170331.00,827.00,3456.00,40.80,-73.93,11.94,7.75,21.62,7.72,0.22,126.20,279.00,480.00,36.70,28.30,8.20


In [7]:
print(len(data['date']))
print(len(data['date'].unique()))

927184
1349


# Identify staions with highest average subscriber daily usage

In [11]:
# Create a dataset for subscribers daily usage by location
sub_by_station= d1[['pickups', 'st_id', 'st_name']].groupby(['st_id', 'st_name']).mean().reset_index()
sub_by_station['stds'] = d1[['pickups', 'st_id', 'st_name']].groupby(['st_id', 'st_name']).std().reset_index()['pickups']
sub_by_station['counts'] = d1[['pickups', 'st_id', 'st_name']].groupby(['st_id', 'st_name']).size().reset_index()[0]

highest10_by_mean_sub  = sub_by_station[sub_by_station['counts'] >= 100].sort_values('pickups', ascending=False).head(10)
lowest10_by_mean_sub = sub_by_station[sub_by_station['counts'] >= 100].sort_values('pickups', ascending=True).head(10)

In [12]:
len(sub_by_station)

667

In [13]:
sub_by_station['counts'].describe().round(0)

count     667.0
mean      810.0
std       503.0
min         1.0
25%       229.0
50%       578.0
75%      1334.0
max      1349.0
Name: counts, dtype: float64

In [14]:
# Top 10 bike stations for subscribers
highest10_by_mean_sub

,st_id,st_name,pickups,stds,counts
294,519,Pershing Square N,292.737194,203.836791,1347
296,521,8 Ave & W 31 St,247.178677,177.655501,1013
508,3263,Cooper Square & E 7 St,238.551724,135.693469,261
273,497,E 17 St & Broadway,222.685034,100.149723,1343
213,435,W 21 St & 6 Ave,222.197026,93.873652,1345
83,293,Lafayette St & E 8 St,219.636812,106.824420,1217
481,3230,Penn Station Valet,214.724638,106.516961,345
184,402,Broadway & E 22 St,188.897626,102.494079,1348
79,285,Broadway & E 14 St,187.233457,78.923310,1345
502,3255,8 Ave & W 31 St,184.226230,83.531459,305


In [15]:
# Bottom 10 bike stations for subscribers
lowest10_by_mean_sub

,st_id,st_name,pickups,stds,counts
474,3219,NYCBS Depot - STY,1.769784,0.911414,139
322,2005,Railroad Ave & Kay Ave,2.538238,1.570297,1033
566,3342,Pioneer St & Richards St,3.069892,1.779905,186
551,3326,Clinton St & Centre St,3.129032,2.119285,186
615,3394,Columbia St & W 9 St,3.211538,3.998402,156
555,3330,Henry St & Bay St,3.427711,2.496825,166
614,3393,Richards St & Delavan St,3.532995,2.093522,197
616,3395,Henry St & W 9 St,3.843434,2.330711,198
341,3044,Albany Ave & Fulton St,3.863039,2.253236,533
558,3333,Columbia St & Lorraine St,4.043956,3.184604,182


In [16]:
# Create a dataset for non-subscribers daily usage by location
nonsub_by_station= d2[['pickups', 'st_id', 'st_name']].groupby(['st_id', 'st_name']).mean().reset_index()
nonsub_by_station['stds'] = d2[['pickups', 'st_id', 'st_name']].groupby(['st_id', 'st_name']).std().reset_index()['pickups']
nonsub_by_station['counts'] = d2[['pickups', 'st_id', 'st_name']].groupby(['st_id', 'st_name']).size().reset_index()[0]

highest10_by_mean_nonsub  = nonsub_by_station[nonsub_by_station['counts'] >= 100].sort_values('pickups', ascending=False).head(10)
lowest10_by_mean_nonsub = nonsub_by_station[nonsub_by_station['counts'] >= 100].sort_values('pickups', ascending=True).head(10)

In [17]:
len(nonsub_by_station)

657

In [18]:
nonsub_by_station['counts'].describe().round(0)

count     657.0
mean      588.0
std       421.0
min         1.0
25%       157.0
50%       489.0
75%      1026.0
max      1261.0
Name: counts, dtype: float64

In [19]:
# Top 10 bike stations for non-subscribers
highest10_by_mean_nonsub

,st_id,st_name,pickups,stds,counts
323,2006,Central Park S & 6 Ave,89.953212,84.182956,1261
169,387,Centre St & Chambers St,66.829032,64.458067,1240
435,3143,5 Ave & E 78 St,60.123016,57.269243,504
76,281,Grand Army Plaza & Central Park S,59.472625,54.725390,1242
429,3137,5 Ave & E 73 St,56.392857,55.022139,504
505,3282,5 Ave & E 88 St,55.815315,55.315362,222
456,3165,Central Park West & W 72 St,52.835391,53.477172,486
205,426,West St & Chambers St,51.807914,52.685694,1213
289,514,12 Ave & W 40 St,49.744787,54.272698,1199
459,3168,Central Park West & W 85 St,44.174393,45.775501,453


In [20]:
# Bottom 10 bike stations for non-subscribers
lowest10_by_mean_nonsub

,st_id,st_name,pickups,stds,counts
322,2005,Railroad Ave & Kay Ave,1.483051,0.792411,118
343,3049,Cambridge Pl & Gates Ave,1.798658,1.257302,149
472,3221,47 Ave & 31 St,1.868421,1.435981,152
416,3123,31 St & Thomson Ave,1.891566,1.330409,166
352,3059,Pulaski St & Marcus Garvey Blvd,1.923729,1.275387,118
363,3070,McKibbin St & Manhattan Ave,1.931507,1.397819,146
339,3044,Albany Ave & Fulton St,1.961290,1.278615,155
492,3249,Verona Pl & Fulton St,1.975000,1.305853,120
354,3061,Throop Ave & Myrtle Ave,2.000000,1.324042,163
388,3095,Graham Ave & Herbert St,2.009259,1.503456,216
